In [1]:
from google.colab import drive
drive.mount('/gdive')

Drive already mounted at /gdive; to attempt to forcibly remount, call drive.mount("/gdive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/gdive/MyDrive/Prove/Valutazione.csv', encoding = 'utf-8')
df

,Frasi,B,M,Media,Valutazione
0,"""È un campo di concentramento"". La sparata di ...",5,5.0,5,5.0
1,"A 87 anni, probabilmente la più anziana del Ve...",0,NaN,0,0.0
2,A condurre la serata l’attrice e presentatrice...,0,NaN,0,0.0
3,A partire da questa mattina presto diverse zon...,0,0.0,0,0.0
4,A perdere la vita è stato un 19enne di origine...,2,0.0,1,1.0
...,...,...,...,...,...
436,Urge ridare ai giovani esempi sani sul rispett...,4,NaN,4,4.0
437,Viene dedicata alla memoria delle giornaliste ...,0,NaN,0,0.0
438,Vista l’età il gip l’ha mandato ai domiciliari...,0,NaN,0,0.0
439,"Vista la resistenza della giovane, l’uomo era ...",0,NaN,0,0.0


In [3]:
df.Frasi
sent = df.Frasi .to_list()

In [4]:
import re #espressioni regolari (modulo)
import nltk #elaborazione linguaggio naturale (libreria)
from nltk.corpus import stopwords #parole comuni da ignorare

In [5]:
nltk.download('stopwords') #la prima volta è necessario scaricare il pacchetto

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop_words = set(stopwords.words('italian')) #prendo le stopwords in italiano e rendo set perchè più meneggevole

In [7]:
#definisco una funzione per pulire il testo
def clean_text(text):
  text = text.lower() #per rendere tutto in minuscolo
  text = re.sub(r'[^a-zà-ù]', '', text) #rimuove tutto tranne lettere italiane minuscole
  tokens = text.split() #divido in tokens la frase
  tokens = [word for word in tokens if word not in stop_words] #rimuove tutte le parole presenti nel set stop_words
  return ' '.join(tokens) #restituisce frase pulita

df['frasi_pulite'] = df['Frasi'].apply(clean_text) #applico la funzione alla colonna frasi


In [8]:
X = df['frasi_pulite']
y = df['Valutazione']

from sklearn.model_selection import train_test_split #importo la funzione per suddividere il dataset in trein e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #random lo fisso in modo che sia riproducibile perche le frasi vengono mescolate prima di essere separati

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer #per trasformare le frasi in vettori e poi farne una matrice

vectorizer = TfidfVectorizer(max_features=1000) #l'argomento serve a limitare il numero di parole per non avere una matrice troppo grande e rischiare di avere overfitting

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)  #trasformo la x_train e x_test in vettori

#non è necessario trasformare anche la colonna dei punteggi (Y) in quanto sono già valori numerici

In [10]:
from sklearn.linear_model import LinearRegression  #importo il modello da addestrare, scelgo questo perchè serve per predirre un numero lineare

model = LinearRegression() #creo il modello
model.fit(X_train_vec, y_train) #uso i dati di train per addestrare il modello

LinearRegression()

In [11]:
from sklearn.metrics import mean_squared_error, r2_score #mi servono per valutare il modello (errore quadratratico medio e quanto copre la variabilità dei dati)


predicts = model.predict(X_test_vec)
error = mean_squared_error(y_test, predicts)
r2 = r2_score(y_test, predicts)
print(f'Errore quadratico medio: {error}')
print(f'R2 score: {r2}')

Errore quadratico medio: 3.5874105732092696
R2 score: -0.00205868466510184


In [12]:
!pip install gradio[oauth]==4.44 #mi installa la versione 4.44 di gradio per fare interfacce

In [13]:
import gradio as gr #importo la libreria per fare interfacce
import numpy as np #serve per arrotondare il numero che ci restituirà

In [14]:
#definisco una funzione che prenda in input una nuova frase e ne predica il punteggio con il modello addestrato

def predici_bias(frase):
    fr_clean = clean_text(frase)
    fr_vett = vectorizer.transform([fr_clean])
    punteggio = model.predict(fr_vett)[0] #applico il modello alla frase pulita e resa vettore
    punt_arrotondato = np.round(punteggio, 1) #arrotondo il valore del bias
    return f"Punteggio di bias stimato: {punt_arrotondato} (da 0 a 5)"


In [15]:
#realizzo l'interfaccia con Gradio

with gr.Blocks() as interfaccia:
  titolo = gr.Markdown("""
    # Valutazione Bias
    Inserisci una frase e il modello stimerà quanto è influenzata da bias cognitivi (da 0 a 5)
  """)
  with gr.Row():
    with gr.Column():
      input = gr.Textbox(label="Frase" , placeholder="Inserisci la frase da valutare")
      with gr.Row():
        cancella_btn = gr.ClearButton(input)
        valuta_btn = gr.Button("Valuta" , variant="primary")
    with gr.Column():
      output = gr.Textbox(label="Punteggio")
      valuta_btn.click(fn=predici_bias, inputs=input, outputs=output)
  #with gr.Row():
    #with gr.Column():
      #intestazione = gr.Markdown("""
        ## Valutazioni passate
     # """)
      #tabella = gr.DataFrame(headers=["Frasi","Valutazioni"], col_count=2)

interfaccia.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://43786fec90733fab6f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
